In [9]:
%pip install nest_asyncio fastapi uvicorn

Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip install -U pip setuptools wheel
%pip install -U spacy
%python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [11]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [12]:
outline_sections = [
    "It took me eighteen years to realize what an extraordinary influence my mother has been on my life.",
    "Her mother's enthusiasm for learning.",
    "Learning through travel by using the example of a trip to Greece.",
    "While I treasure the various worlds my mother has opened to me abroad, my life has been equally transformed by what she has shown me just two miles from my house.",
    "Her mother's dedication to the community.",
    "Her multiple volunteer activities such as helping at the local soup kitchen.",
    "Everything that my mother has ever done has been overshadowed by the thought behind it.",
    "She has enriched my life with her passion for learning, and changed it with her devotion to humanity.",
    "Next year, I will find a new home miles away. However, my mother will always be by my side."
]
outline_docs = [nlp(outline_sen) for outline_sen in outline_sections]


In [13]:
example = [
"It took me eighteen years to realize what an extraordinary influence my mother has been on my life.",
"She's the kind of person who has thoughtful discussions about which artist she would most want to have her portrait painted by (Sargent), the kind of mother who always has time for her four children, and the kind of community leader who has a seat on the board of every major project to assist Washington's impoverished citizens.",
"Growing up with such a strong role model, I developed many of her enthusiasms.I not only came to love the excitement of learning simply for the sake of knowing something new, but I also came to understand the idea of giving back to the community in exchange for a new sense of life, love, and spirit.",
"My mother's enthusiasm for learning is most apparent in travel.",
"Despite the fact that we were traveling with fourteen-month-old twins, we managed to be at each ruin when the site opened at sunrise.",
"I vividly remember standing in an empty amphitheatre pretending to be an ancient tragedian, picking out my favorite sculpture in the Acropolis museum, and inserting our family into modified tales of the battle at Troy.",
"Eight years and half a dozen passport stamps later I have come to value what I have learned on these journes about global history, politics and culture, as well as my family and myself.",
"I was nine years old when my family visited Greece. Every night for three weeks before the trip, my older brother Peter and I sat with my mother on her bed reading Greek myths and taking notes on the Greek Gods.",
'''While I treasure the various worlds my mother has opened to me abroad, my life has been equally transformed by what she has shown me just two miles from my house.''',
'''As a ten year old, I often accompanied my mother to (name deleted), a local soup kitchen and children's center.''',
"While she attended meetings, I helped with the Summer Program by chasing children around the building and performing magic tricks.",
'''Having finally perfected the "floating paintbrush" trick, I began work as a full time volunteer with the five and six year old children last June.''',
"It is here that I met Jane Doe, an exceptionally strong girl with a vigor that is contagious.",
"At the end of the summer, I decided to continue my work at (name deleted) as Jane's tutor.Although the position is often difficult, the personal rewards are beyond articulation.",
"In the seven years since I first walked through the doors of (name deleted), I have learned not only the idea of giving to others, but also of deriving from them a sense of spirit.",
"Everything that my mother has ever done has been overshadowed by the thought behind it.",
"While the raw experiences I have had at home and abroad have been spectacular, I have learned to truly value them by watching my mother.",
"She has enriched my life with her passion for learning, and changed it with her devotion to humanity.",
"In her endless love of everything and everyone she is touched by, I have seen a hope and life that is truly exceptional.",
"Next year, I will find a new home miles away. However, my mother will always be by my side."
]
text_docs = [nlp(text_sen) for text_sen in example]
similarities = []
for index, par in enumerate(outline_docs):
    print(index, par)
    similarities.append([outline_docs[index].similarity(sen) for sen in text_docs])

similarities

0 It took me eighteen years to realize what an extraordinary influence my mother has been on my life.
1 Her mother's enthusiasm for learning.
2 Learning through travel by using the example of a trip to Greece.
3 While I treasure the various worlds my mother has opened to me abroad, my life has been equally transformed by what she has shown me just two miles from my house.
4 Her mother's dedication to the community.
5 Her multiple volunteer activities such as helping at the local soup kitchen.
6 Everything that my mother has ever done has been overshadowed by the thought behind it.
7 She has enriched my life with her passion for learning, and changed it with her devotion to humanity.
8 Next year, I will find a new home miles away. However, my mother will always be by my side.


C:\Users\Young Kwang Choi\AppData\Local\Temp\ipykernel_7316\119260243.py:27: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarities.append([outline_docs[index].similarity(sen) for sen in text_docs])


[[1.0,
  0.5015854505625235,
  0.42948603427839,
  0.33601756624973406,
  0.41739696865611775,
  0.45764813528479403,
  0.3665081898964689,
  0.568412047140545,
  0.7364236136789247,
  0.23852162363493362,
  0.40210350587660365,
  0.2982752323992235,
  0.5163008611573663,
  0.4996415096260461,
  0.37432337246419084,
  0.5111362150867655,
  0.5152168979672354,
  0.4774554537206406,
  0.5433872971494835,
  0.40086949351124374],
 [0.30504830155576484,
  0.3884681561384705,
  0.40378763127627393,
  0.8195845605350746,
  0.2785591536477794,
  0.35885465402407885,
  0.28126141285570133,
  0.46018389780756885,
  0.21991356759716185,
  0.4798322541743295,
  0.40138109283026974,
  0.1534324153756725,
  0.14010781592752347,
  0.46936321037409484,
  0.28676931430331576,
  0.19379623933484091,
  0.10146540575886975,
  0.5098585652222948,
  0.24488369914398372,
  0.2251487722506775],
 [0.36376850372477537,
  0.44839873603690233,
  0.7588979606278096,
  0.4889747979996867,
  0.5151857590307564,
  0.

In [69]:
from typing import Optional 
from fastapi import FastAPI, Response
from fastapi.middleware.cors import CORSMiddleware 
from fastapi.responses import HTMLResponse, PlainTextResponse
from fastapi.staticfiles import StaticFiles
import json


app = FastAPI() 

origins = [ 
    "http://localhost", 
    "http://localhost:8080", 
] 

app.add_middleware( 
    CORSMiddleware, 
    allow_origins=origins, 
    allow_credentials=True, 
    allow_methods=["*"], 
    allow_headers=["*"], 
) 

@app.get("/", response_class=HTMLResponse) 
def read_root(): 
    return open("index.html").read()

app.mount("/static", StaticFiles(directory="static"), name="static")

@app.get("/items/{item_id}") 
def read_item(item_id: int, q: Optional[str] = None): 
    return {"item_id": item_id, "q": q}

@app.get("/item/match")
async def read_items():
    data = (similarities)
    return data

@app.get("/item/logprobs")
async def read_items():
    with open('./value.txt', 'r') as f:
        my_list = [float(line.strip()) for line in f]
    return my_list

    

@app.get("/item")
async def read_items():
    # data = "My mother's enthusiasm for learning is most apparent in travel. Despite the fact that we were traveling with fourteen-month-old twins, we managed to be at each ruin when the site opened at sunrise. I vividly remember standing in an empty amphitheatre pretending to be an ancient tragedian, picking out my favorite sculpture in the Acropolis museum, and inserting our family into modified tales of the battle at Troy. I was nine years old when my family visited Greece. Every night for three weeks before the trip, my older brother Peter and I sat with my mother on her bed reading Greek myths and taking notes on the Greek Gods. Eight years and half a dozen passport stamps later I have come to value what I have learned on these journes about global history, politics and culture, as well as my family and myself."

    data = (
            {
                "text":"I vividly remember standing in an empty amphitheatre pretending to be an ancient tragedian, picking out my favorite sculpture in the Acropolis museum, and inserting our family into modified tales of the battle at Troy. ",
                "comment":"",
                "new-order":"0"
                
            },
            {
                "text":"I was nine years old when my family visited Greece. ",
                "comment":"",
                "new-order":"2"
                
            },
            {
                "text":"Every night for three weeks before the trip, my older brother Peter and I sat with my mother on her bed reading Greek myths and taking notes on the Greek Gods. ",
                "comment":"",
                "new-order":"3"
                
            },
            {
                "text":"Eight years and half a dozen passport stamps later I have come to value what I have learned on these journes about global history, politics and culture, as well as my family and myself. ",
                "comment":"",
                "new-order":"1"
            }
    )
    return data

import nest_asyncio 
import uvicorn 

nest_asyncio.apply() 
uvicorn.run(app, port=8000) 

INFO:     Started server process [7316]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:56541 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:56541 - "GET /styles/main_page.css HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:57527 - "GET /item/logprobs HTTP/1.1" 200 OK
